In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'gender_submission.csv', 'test.csv']


In [2]:
train_df=pd.read_csv("../input/train.csv")
test_df=pd.read_csv("../input/test.csv")

In [3]:
print(train_df.info())
print("--"*20)
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non

In [4]:
X=train_df.drop("Survived",axis=1)
Y=train_df["Survived"]

In [5]:
combined_df=pd.concat([X,test_df])

In [6]:
print(combined_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 122.7+ KB
None


# Data Cleaning
* Correct
* Complete
* Create
* Convert

In [7]:
from sklearn.impute import SimpleImputer

Age_imputer=SimpleImputer(strategy="median")
Fare_imputer=SimpleImputer(strategy="median")
Embarked_imputer=SimpleImputer(strategy="most_frequent")


combined_df["Age"]=Age_imputer.fit_transform(combined_df[["Age"]])
combined_df["Fare"]=Age_imputer.fit_transform(combined_df[["Fare"]])
combined_df["Embarked"]=Embarked_imputer.fit_transform(combined_df[["Embarked"]])

In [8]:
combined_df.drop(["Cabin"],axis=1,inplace=True)

In [9]:
combined_df["Family_size"]=combined_df["SibSp"]+combined_df["Parch"]+1
combined_df["Is_alone"]=int(0)
combined_df["Is_alone"][combined_df["Family_size"]==1]=int(1)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
combined_df['Title']=combined_df['Name'].str.split(",", expand=True)[1].str.split(".", expand=True)[0]

In [11]:
combined_df["Fare_bin"]=pd.qcut(combined_df["Fare"],4)
combined_df["Age_bin"]=pd.cut(combined_df["Age"].astype(int),4)

In [12]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()


combined_df["Sex_encode"]=encoder.fit_transform(combined_df["Sex"])
combined_df["Age_encode"]=encoder.fit_transform(combined_df["Age_bin"])
combined_df["Fare_encode"]=encoder.fit_transform(combined_df["Fare_bin"])
combined_df["Title_encode"]=encoder.fit_transform(combined_df["Title"])
combined_df["Embarked_encode"]=encoder.fit_transform(combined_df["Embarked"])

In [13]:
combined_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Family_size,Is_alone,Title,Fare_bin,Age_bin,Sex_encode,Age_encode,Fare_encode,Title_encode,Embarked_encode
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,2,0,Mr,"(-0.001, 7.896]","(20.0, 40.0]",1,1,0,12,2
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,2,0,Mrs,"(31.275, 512.329]","(20.0, 40.0]",0,1,3,13,0
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,1,1,Miss,"(7.896, 14.454]","(20.0, 40.0]",0,1,1,9,2
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,2,0,Mrs,"(31.275, 512.329]","(20.0, 40.0]",0,1,3,13,2
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,1,1,Mr,"(7.896, 14.454]","(20.0, 40.0]",1,1,1,12,2


In [14]:
# Features to be used
df_final=combined_df[["Pclass","Family_size","Is_alone","Sex_encode","Age_encode","Fare_encode","Embarked_encode"]]
X_train=df_final[0:891]
X_test=df_final[891:]

In [15]:
X_train.head()

,Pclass,Family_size,Is_alone,Sex_encode,Age_encode,Fare_encode,Embarked_encode
0,3,2,0,1,1,0,2
1,1,2,0,0,1,3,0
2,3,1,1,0,1,1,2
3,1,2,0,0,1,3,2
4,3,1,1,1,1,1,2


# Model Creation

In [16]:
from sklearn import ensemble, neural_network , neighbors , svm , linear_model ,naive_bayes ,tree , gaussian_process,discriminant_analysis
from xgboost import XGBClassifier, XGBRFClassifier

In [17]:
#Machine Learning Algorithms
MLA=[
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    linear_model.LogisticRegressionCV(),
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #Nearest Neighbor
    neighbors.KNeighborsClassifier(),
    
    #SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    svm.LinearSVC(),
    
    #Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    #Neural Network
    neural_network.MLPClassifier(),
    
    #xgboost
    XGBClassifier(),
    XGBRFClassifier()
]

In [18]:
MLA_compare=["Algorithm_Name","Parameters", "Mean_Accuracy","Standard_Deviation"]
MLA_compare=pd.DataFrame(columns=MLA_compare)

In [19]:
from sklearn.model_selection import cross_val_score

index=0
for algo in MLA:
    clf=algo
    clf.fit(X_train,Y)
    cv=cross_val_score(estimator=algo,X=X_train,y=Y,cv=10,n_jobs=-1)
    cv_mean=cv.mean()
    cv_std=cv.std()
    MLA_compare.loc[index,"Algorithm_Name"]=algo.__class__.__name__
    MLA_compare.loc[index,"Parameters"]=str(algo.get_params())
    MLA_compare.loc[index,"Mean_Accuracy"]=cv_mean
    MLA_compare.loc[index,"Standard_Deviation"]=cv_std
    index+=1

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWar

In [20]:
MLA_compare

,Algorithm_Name,Parameters,Mean_Accuracy,Standard_Deviation
0,AdaBoostClassifier,"{'algorithm': 'SAMME.R', 'base_estimator': Non...",0.789008,0.0243432
1,BaggingClassifier,"{'base_estimator': None, 'bootstrap': True, 'b...",0.810483,0.0468715
2,ExtraTreesClassifier,"{'bootstrap': False, 'class_weight': None, 'cr...",0.805938,0.0405506
3,GradientBoostingClassifier,"{'criterion': 'friedman_mse', 'init': None, 'l...",0.813754,0.0284525
4,RandomForestClassifier,"{'bootstrap': True, 'class_weight': None, 'cri...",0.809334,0.0421505
5,GaussianProcessClassifier,"{'copy_X_train': True, 'kernel': None, 'max_it...",0.802542,0.0378785
6,LogisticRegressionCV,"{'Cs': 10, 'class_weight': None, 'cv': 'warn',...",0.790195,0.0171419
7,PassiveAggressiveClassifier,"{'C': 1.0, 'average': False, 'class_weight': N...",0.697219,0.115759
8,RidgeClassifierCV,"{'alphas': array([ 0.1, 1. , 10. ]), 'class_w...",0.800232,0.0198522
9,SGDClassifier,"{'alpha': 0.0001, 'average': False, 'class_wei...",0.762177,0.0615551


## Fine Tuning Models by tuning Hyper parameters

In [21]:
selected_model=svm.NuSVC(probability=True)
selected_model.fit(X_train,Y)

/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


NuSVC(cache_size=200, class_weight=None, coef0=0.0,
      decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
      kernel='rbf', max_iter=-1, nu=0.5, probability=True, random_state=None,
      shrinking=True, tol=0.001, verbose=False)

In [22]:
# Parameters=[{"base_score":[0.1,0.2,0.3,0.5,0.6,0.9,1,2],
#              "gamma":[0,0.1,0.2,0.3,0.4,0.5,1],
#              "learning_rate":[0,0.1,0.2,0.3,0.4,1],
#             }]

In [23]:
# from sklearn.model_selection import GridSearchCV
# grid=GridSearchCV(estimator=selected_model,param_grid=Parameters,n_jobs=-1,cv=10,scoring="accuracy")
# result=grid.fit(X=X_train,y=Y)

In [24]:
# best_accuracy=result.best_score_
# best_parameters=result.best_params_

In [25]:
y_pred=selected_model.predict(X_test)
y_pred=pd.DataFrame({"PassengerId":test_df["PassengerId"],"Survived":y_pred})
y_pred.to_csv("Submit.csv",index=False)

In [26]:
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S
